In [1]:
import pandas as pd
from tqdm import tqdm
import obspy
import numpy as np
from obspy.core.utcdatetime import UTCDateTime
import sys
import h5py 
sys.path.append("/data/wsd01/pnwstore/")

import matplotlib.pyplot as plt
from obspy.core.utcdatetime import UTCDateTime
from pnwstore.catalog import QuakeClient
from pnwstore.station import StationClient

from collections import Counter

In [2]:
quakeclient = QuakeClient("cascadia", "pnwstore")
stationclient = StationClient("cascadia", "pnwstore")
pd.set_option('display.max_columns', None)

In [3]:
df_path = "/data/whd01/yiyu_data/PNWML/extract/norcal_metadata.csv"

In [4]:
df = pd.read_csv(df_path)
df

,event_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,source_magnitude,source_magnitude_type,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,trace_P_arrival_sample,trace_P_arrival_uncertainty_s,trace_P_polarity,trace_P_onset
0,uw61899326,2022-12-20T13:35:04.490000Z,40.549,-124.400,earthquake,8.88,3.4,Mh,31.61,11.17,BK,HH,YBH,00,TBA,TBA,TBA,"bucket1$0,:3,:25001",100,2022-12-20T13:34:14.490000Z,7929,0.06,positive,impulsive
1,uw61899326,2022-12-20T13:35:04.490000Z,40.549,-124.400,earthquake,8.88,3.4,Mh,31.61,11.17,NC,HH,KSXB,--,TBA,TBA,TBA,"bucket1$1,:3,:25001",100,2022-12-20T13:34:14.490000Z,7365,0.03,positive,impulsive
2,uw61899326,2022-12-20T13:35:04.490000Z,40.549,-124.400,earthquake,8.88,3.4,Mh,31.61,11.17,UO,HH,CAVE,--,42.1209,-123.571,440.00,"bucket1$2,:3,:25001",100,2022-12-20T13:34:14.490000Z,7856,0.03,positive,impulsive
3,uw61899326,2022-12-20T13:35:04.490000Z,40.549,-124.400,earthquake,8.88,3.4,Mh,31.61,11.17,UO,HH,SISQ,--,42.2754,-123.632,395.00,"bucket1$3,:3,:25001",100,2022-12-20T13:34:14.490000Z,8084,0.03,negative,impulsive
4,uw61899326,2022-12-20T13:35:04.490000Z,40.549,-124.400,earthquake,8.88,3.4,Mh,31.61,11.17,PB,EH,B040,--,41.8308,-122.421,819.50,"bucket1$4,:3,:25001",100,2022-12-20T13:34:14.490000Z,8306,0.03,positive,impulsive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,uw61891007,2022-12-22T11:49:57.230000Z,40.408,-124.268,earthquake,24.48,3.8,Mh,1.78,1.66,UO,HH,DFAZ,--,43.2406,-122.109,1948.00,"bucket1$600,:3,:25001",100,2022-12-22T11:49:07.230000Z,9945,0.30,undecidable,emergent
601,uw61891007,2022-12-22T11:49:57.230000Z,40.408,-124.268,earthquake,24.48,3.8,Mh,1.78,1.66,PB,EH,B031,--,43.6643,-123.397,71.00,"bucket1$601,:3,:25001",100,2022-12-22T11:49:07.230000Z,9946,0.15,undecidable,emergent
602,uw61891007,2022-12-22T11:49:57.230000Z,40.408,-124.268,earthquake,24.48,3.8,Mh,1.78,1.66,PB,EH,B032,--,43.668,-123.392,63.80,"bucket1$602,:3,:25001",100,2022-12-22T11:49:07.230000Z,9930,0.06,positive,impulsive
603,uw61891007,2022-12-22T11:49:57.230000Z,40.408,-124.268,earthquake,24.48,3.8,Mh,1.78,1.66,UO,HH,DRAN,--,43.6972,-123.349,140.60,"bucket1$603,:3,:25001",100,2022-12-22T11:49:07.230000Z,9989,0.06,positive,impulsive


## check location 

In [5]:
df['station_location_code'].unique()

array(['00', '--', '03', '01'], dtype=object)

In [33]:
df.iloc[df[pd.isna(df['station_location_code'])].index, 13] = '--'
df.iloc[df[df['station_location_code'] == 0.0].index, 13] = "00"
df.iloc[df[df['station_location_code'] == 3.0].index, 13] = "03"
df.iloc[df[df['station_location_code'] == 1.0].index, 13] = "01"

In [34]:
df.to_csv(df_path, sep = ',', index=False)

In [63]:
df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv")
# df.iloc[df[(df['station_code'] == 'EPH2') & (df['trace_channel'] == 'EH')].index, 2] = "HH"
# df.to_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv", sep = ',', index=False)

## add lat lon elevation

In [6]:
df = pd.read_csv(df_path)

In [7]:
error = []
for idx, i in tqdm(df.iterrows(), total = len(df)):
#     if df.iloc[idx, 14] == "TBA":
    if True:
        network = df.iloc[idx, 10]
        station = df.iloc[idx, 12]
        channel = df.iloc[idx, 11] + "Z"
        time = UTCDateTime(df.iloc[idx, 19])
        result = stationclient.query(network = network, station = station, channel = channel, time = time)
        
        try:
            assert len(result) >= 1
            df.iloc[idx, 14] = result.iloc[0, 5]         # latitude
            df.iloc[idx, 15] = result.iloc[0, 6]         # longitude
            df.iloc[idx, 16] = result.iloc[0, 7]         # elevation
            
        except:
            print(idx)
            error.append([idx, i])

  0%|          | 3/605 [00:00<00:27, 21.91it/s]

0
1


  4%|▍         | 24/605 [00:01<00:25, 22.88it/s]

20
21


  6%|▌         | 36/605 [00:01<00:24, 22.96it/s]

...

 95%|█████████▍| 573/605 [00:24<00:01, 23.06it/s]

569
572


 96%|█████████▌| 582/605 [00:25<00:00, 23.02it/s]

579


100%|██████████| 605/605 [00:26<00:00, 22.93it/s]


In [9]:
from obspy.clients.fdsn.client import Client
from obspy.clients.fdsn.header import URL_MAPPINGS
URL_MAPPINGS['NCEDC'] = "https://service.ncedc.org"

In [10]:
ncedc = Client("NCEDC")

In [36]:
error = []
for idx, i in tqdm(df.iterrows(), total = len(df)):
    if df.iloc[idx, 14] == "TBA":
        network = df.iloc[idx, 10]
        station = df.iloc[idx, 12]
        channel = df.iloc[idx, 11] + "Z"
        location = df.iloc[idx, 13]
        time = UTCDateTime(df.iloc[idx, 19])
        s = ncedc.get_stations(network = network, 
                               station = station, 
                               location = location, 
                               channel = channel, 
                               level = 'station')
        
#         try:
        assert len(s) >= 1
        df.iloc[idx, 14] = s[0][0].latitude         # latitude
        df.iloc[idx, 15] = s[0][0].longitude         # longitude
        df.iloc[idx, 16] = s[0][0].elevation         # elevation
            
#         except:
#             print(idx)
#             error.append([idx, i])


  0%|          | 0/605 [00:00<?, ?it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
  0%|          | 1/605 [00:00<07:06,  1.42it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
  0%|          | 2/605 [00:01<06:44,  1.49it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
  3%|▎         | 21/605 [00:02<00:43, 13.50it/s]/home/niyiyu/an

 12%|█▏        | 72/605 [00:17<04:31,  1.96it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 12%|█▏        | 73/605 [00:18<04:48,  1.84it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 12%|█▏        | 74/605 [00:19<05:09,  1.72it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 13%|█▎        | 79/605 [00:19<02:35,  3.38it/s]/hom

 40%|████      | 242/605 [00:36<00:44,  8.09it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 40%|████      | 244/605 [00:38<00:58,  6.14it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 41%|████      | 246/605 [00:38<01:04,  5.55it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/

 61%|██████▏   | 372/605 [00:54<00:13, 17.48it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 62%|██████▏   | 374/605 [00:54<00:17, 13.20it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 63%|██████▎   | 383/605 [00:55<00:16, 13.10it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/

 71%|███████   | 430/605 [01:11<01:27,  2.00it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 71%|███████   | 431/605 [01:12<01:33,  1.86it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 71%|███████▏  | 432/605 [01:12<01:38,  1.76it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 72%|███████▏  | 433/605 [01:13<01:43,  1.66it/s]

 92%|█████████▏| 559/605 [01:28<00:04, 10.65it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 93%|█████████▎| 561/605 [01:30<00:06,  6.40it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/site-packages/obspy/io/stationxml/core.py:96: UserWarning: The StationXML file has version 1.2, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  warnings.warn("The StationXML file has version %s, ObsPy can "
 93%|█████████▎| 562/605 [01:30<00:08,  5.22it/s]/home/niyiyu/anaconda3/envs/seisml/lib/python3.8/